<a href="https://colab.research.google.com/github/iguerrasevillano/Automated_Summaries_Generation_Scientific_Papers/blob/main/Automatization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling tenso

In [31]:
# Libraries
import os
import pandas as pd
import numpy as np
import json

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Keras
import tensorflow as tf

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist

nltk.download('punkt')

# Transformers
from transformers import TFAutoModelForSeq2SeqLM, BartTokenizer, pipeline

# Graph
import networkx as nx
import re

# Time
import time

# AST
import ast

# Current directory
os.getcwd()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'/content'

In [3]:
def similarity(sentence1, sentence2, stopwords=None):
  if stopwords is None:
    stopwords = []
  sentence1 = [w.lower() for w in sentence1]
  sentence2 = [w.lower() for w in sentence2]

  all_words = list(set(sentence1 + sentence2))

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)

  #build the vector for the first sentence
  for word in sentence1:
    if not word in stopwords:
      vector1[all_words.index(word)]+=1

  #build the vector for the second sentence
  for word in sentence2:
    if not word in stopwords:
      vector2[all_words.index(word)]+=1

  norm_vector1 = np.sqrt(np.dot(vector1, vector1))
  norm_vector2 = np.sqrt(np.dot(vector2, vector2))

  if norm_vector1 == 0 or norm_vector2 == 0:
    return 0

  return 1-cosine_distance(vector1, vector2)



def similarity_matrix(sentences, stop_words):
  similarity_matrix = np.zeros((len(sentences), len(sentences)))

  for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
      if idx1 != idx2:
        similarity_matrix[idx1][idx2] = similarity(sentences[idx1], sentences[idx2], stop_words)

  return similarity_matrix



def generate_extractive_summary(sentences, top_n, stop_words):

  summarize_text = []

  # Step1: generate similarity matrix
  sentence_similarity_matrix = similarity_matrix(sentences, stop_words)

  # Step2: Rank sentences in similarity matrix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
  scores = nx.pagerank(sentence_similarity_graph)

  # Step3: sort the rank and place top sentences
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

  # Step4: get the top n number of sentences based on rank
  for i in range(top_n):
    if len(ranked_sentences) > i:
      summarize_text.append(ranked_sentences[i][1])
    else:
      break

  # Step5: reorder the sentences as in the original text
  real_indexes = []
  for sentence in summarize_text:
    if sentence in sentences:
      real_indexes.append(sentences.index(sentence))
    else:
      continue

  sorted_real_indexes = sorted(real_indexes)

  ordered_summarized_text = []
  for i in sorted_real_indexes:
    ordered_summarized_text.append(sentences[i])

  # Step6 : output the summarized version
  return ' '.join(ordered_summarized_text)

In [4]:
# Connect w/ Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [14]:
BASE_PATH = "/content/drive/MyDrive/VIU/TFM/Desarrollo/"
DATA_PATH = BASE_PATH + 'Data/'

documents = os.listdir(DATA_PATH)

In [6]:
# Open the file in read mode
with open(DATA_PATH + 'streamlit_download.txt', 'r') as file:
    # Read the content of the file into a string
    file_content = file.read()

# Print the content
print(file_content)

BERT: Pre-training of Deep Bidirectional Transformers for

Language Understanding

Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova

{jacobdevlin,mingweichang,kentonl,kristout}@google.com

Google AI Language

9
1
0
2

 

y
a
M
4
2

 

 
 
]
L
C
.
s
c
[
 
 

2
v
5
0
8
4
0

.

0
1
8
1
:
v
i
X
r
a

Abstract

We introduce a new language representa-
tion model called BERT, which stands for
Bidirectional Encoder Representations from
Transformers. Unlike recent language repre-
sentation models (Peters et al., 2018a; Rad-
ford et al., 2018), BERT is designed to pre-
train deep bidirectional representations from
unlabeled text by jointly conditioning on both
left and right context in all layers. As a re-
sult, the pre-trained BERT model can be ﬁne-
tuned with just one additional output layer
to create state-of-the-art models for a wide
range of tasks, such as question answering and
language inference, without substantial task-
speciﬁc architecture modiﬁcations.
BERT is conceptually sim

In [7]:
input_text = sent_tokenize(file_content)

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
top_n = 16

extractive_text = generate_extractive_summary(input_text,
                                              top_n,
                                              stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
print(extractive_text)

Unlike recent language repre-
sentation models (Peters et al., 2018a; Rad-
ford et al., 2018), BERT is designed to pre-
train deep bidirectional representations from
unlabeled text by jointly conditioning on both
left and right context in all layers. It obtains new state-of-the-art re-
sults on eleven natural language processing
tasks, including pushing the GLUE score to
80.5% (7.7% point absolute improvement),
MultiNLI accuracy to 86.7% (4.6% absolute
improvement), SQuAD v1.1 question answer-
ing Test F1 to 93.2 (1.5 point absolute im-
provement) and SQuAD v2.0 Test F1 to 83.1
(5.1 point absolute improvement). Introduction

1
Language model pre-training has been shown to
be effective for improving many natural language
processing tasks (Dai and Le, 2015; Peters et al.,
2018a; Radford et al., 2018; Howard and Ruder,
2018). These include sentence-level tasks such as
natural language inference (Bowman et al., 2015;
Williams et al., 2018) and paraphrasing (Dolan
and Brockett, 2005), which

In [15]:
model = 'BART/'

save_path = BASE_PATH + '/Results/TLDR/' + model + 'model_save/' + 'sampling-norep-v3/'
model = TFAutoModelForSeq2SeqLM.from_pretrained(save_path)
print(model.generation_config)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/VIU/TFM/Desarrollo//Results/TLDR/BART/model_save/sampling-norep-v3/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


GenerationConfig {
  "begin_suppress_tokens": [
    170
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "do_sample": true,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 150,
  "min_length": 60,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "repetition_penalty": 1.8,
  "suppress_tokens": [
    1698,
    32687
  ],
  "temperature": 0.5
}



In [24]:
model_name = 'facebook/bart-base'

tokenizer = BartTokenizer.from_pretrained(model_name, errors='ignore')

model_input = tokenizer(extractive_text, max_length=1024, truncation=True, padding=True)

print(model_input)

{'input_ids': [0, 38243, 485, 2777, 2851, 241, 12, 50118, 19530, 1258, 3092, 36, 510, 20413, 4400, 1076, 482, 199, 102, 131, 7312, 12, 50118, 1891, 4400, 1076, 482, 199, 238, 163, 18854, 16, 1887, 7, 1198, 12, 50118, 21714, 1844, 2311, 43606, 337, 30464, 31, 50118, 879, 33480, 196, 2788, 30, 13521, 15053, 15, 258, 50118, 6960, 8, 235, 5377, 11, 70, 13171, 4, 85, 6168, 30759, 92, 194, 12, 1116, 12, 627, 12, 2013, 769, 12, 50118, 29, 34869, 15, 19353, 1632, 2777, 5774, 50118, 90, 40981, 6, 217, 3784, 5, 12209, 9162, 1471, 7, 50118, 2940, 4, 245, 207, 36, 406, 4, 406, 207, 477, 7833, 3855, 238, 50118, 46064, 487, 27049, 8611, 7, 8162, 4, 406, 207, 36, 306, 4, 401, 207, 7833, 50118, 37362, 1757, 238, 208, 12444, 2606, 748, 134, 4, 134, 864, 1948, 12, 50118, 154, 4500, 274, 134, 7, 8060, 4, 176, 36, 134, 4, 245, 477, 7833, 4356, 12, 50118, 4892, 548, 1757, 43, 8, 208, 12444, 2606, 748, 176, 4, 288, 4500, 274, 134, 7, 8101, 4, 134, 50118, 1640, 245, 4, 134, 477, 7833, 3855, 322, 24474, 50118

In [28]:
print(model_input['input_ids'])

[0, 38243, 485, 2777, 2851, 241, 12, 50118, 19530, 1258, 3092, 36, 510, 20413, 4400, 1076, 482, 199, 102, 131, 7312, 12, 50118, 1891, 4400, 1076, 482, 199, 238, 163, 18854, 16, 1887, 7, 1198, 12, 50118, 21714, 1844, 2311, 43606, 337, 30464, 31, 50118, 879, 33480, 196, 2788, 30, 13521, 15053, 15, 258, 50118, 6960, 8, 235, 5377, 11, 70, 13171, 4, 85, 6168, 30759, 92, 194, 12, 1116, 12, 627, 12, 2013, 769, 12, 50118, 29, 34869, 15, 19353, 1632, 2777, 5774, 50118, 90, 40981, 6, 217, 3784, 5, 12209, 9162, 1471, 7, 50118, 2940, 4, 245, 207, 36, 406, 4, 406, 207, 477, 7833, 3855, 238, 50118, 46064, 487, 27049, 8611, 7, 8162, 4, 406, 207, 36, 306, 4, 401, 207, 7833, 50118, 37362, 1757, 238, 208, 12444, 2606, 748, 134, 4, 134, 864, 1948, 12, 50118, 154, 4500, 274, 134, 7, 8060, 4, 176, 36, 134, 4, 245, 477, 7833, 4356, 12, 50118, 4892, 548, 1757, 43, 8, 208, 12444, 2606, 748, 176, 4, 288, 4500, 274, 134, 7, 8101, 4, 134, 50118, 1640, 245, 4, 134, 477, 7833, 3855, 322, 24474, 50118, 50118, 134, 

In [29]:
ids = model.generate([model_input['input_ids']],
                     generation_config=model.generation_config,
                     seed=42)

In [30]:
tokenizer.batch_decode(ids, skip_special_tokens=True)

['BERT is a novel pre-training model for natural language processing that outperforms existing multi-sentation models (Peters et al., 2018a; Rad-ford et al, 2018), BERT is designed to pre-train deep bidirectional representations from non-unlabeled text by jointly conditioning on both the context-left and right context in all layers. It obtains new state-of-the-art re-conditioning results for several NLP task-specific sub-tasks, including pushing the GLUE score to >80.5% (7.7% point absolute improvement), Multi-MultiNLI accuracy to 86.3% (4.6% absolute improvement/improvement), S']

In [41]:
summarizer = pipeline('summarization',
                      model=model,
                      tokenizer=tokenizer,
                      framework ='tf')

summarizer(extractive_text)

InvalidArgumentError: Exception encountered when calling layer 'embed_positions' (type TFBartLearnedPositionalEmbedding).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1024] = 1026 is not in [0, 1026) [Op:ResourceGather] name: 

Call arguments received by layer 'embed_positions' (type TFBartLearnedPositionalEmbedding):
  • input_shape=['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)']
  • past_key_values_length=0
  • position_ids=None

In [40]:
summarizer(extractive_text)

InvalidArgumentError: Exception encountered when calling layer 'embed_positions' (type TFBartLearnedPositionalEmbedding).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1024] = 1026 is not in [0, 1026) [Op:ResourceGather] name: 

Call arguments received by layer 'embed_positions' (type TFBartLearnedPositionalEmbedding):
  • input_shape=['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(), dtype=int32)']
  • past_key_values_length=0
  • position_ids=None